# Carga de librerias 
En este fragmento de código, se importan varias librerías esenciales para el procesamiento de imágenes y la implementación de aprendizaje automático. cv2 es importada para el manejo de imágenes, os para la manipulación de rutas y archivos en el sistema operativo, numpy para la gestión de estructuras de datos numéricos, y pickle para guardar y cargar objetos en Python. Además, se incluyen partes del módulo sklearn, específicamente para la creación y evaluación de un modelo de Máquina de Soporte Vectorial (SVM), junto con herramientas para dividir datos y medir la precisión del modelo.

In [ ]:
import os
import cv2 as cv
import numpy as np
import pickle
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Configuración de rutas para el procesamiento de imágenes

En este fragmento de código, se configuran las rutas base para acceder a las imágenes que se utilizarán en el procesamiento. Se define ruta_base como la ubicación principal donde se almacenan los datos. A partir de esta ruta base, se especifican dos subdirectorios: positivas_dir para las imágenes clasificadas como positivas y negativas_dir para las imágenes clasificadas como negativas. Esto permite organizar los datos de manera eficiente para su posterior manipulación y análisis.

In [ ]:
ruta_base = r'C:\\Users\\Janin\\Documents\\IA\\datasetWaldo'
positivas_dir = os.path.join(ruta_base, 'positivo')
negativas_dir = os.path.join(ruta_base, 'negativo')

# Extracción de características HOG para procesamiento de imágenes

Este fragmento de código define una función para extraer características utilizando el descriptor de Histogramas de Gradientes Orientados (HOG), una técnica popular en visión por computadora para la detección de objetos. La función convierte una imagen a escala de grises, la redimensiona a un tamaño predeterminado y luego calcula su descriptor HOG. Las características extraídas se aplanan en un solo vector para su fácil manipulación. Además, se inicializan dos listas, data y labels, que almacenarán estos vectores de características y sus etiquetas correspondientes, respectivamente.

In [ ]:
def extract_features(img, size=(64, 128)):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    resized = cv.resize(gray, size)

    hog = cv.HOGDescriptor(_winSize=(size[0] // 8 * 8, size[1] // 8 * 8),
                           _blockSize=(16, 16),
                           _blockStride=(8, 8),
                           _cellSize=(8, 8),
                           _nbins=9)
    h = hog.compute(resized)
    return h.flatten()

data = []
labels = []

# Procesamiento de imágenes y entrenamiento de SVM para clasificación

Este fragmento de código carga imágenes de dos directorios, uno con imágenes positivas y otro con negativas, extrae características usando el descriptor HOG, y las etiqueta como 1 (positiva) o 0 (negativa). Las características se almacenan en un array de Numpy junto con sus etiquetas. Posteriormente, se divide el conjunto de datos en entrenamiento y prueba, y se entrena un modelo SVM con kernel lineal. Finalmente, se evalúa la precisión del modelo y se guarda el modelo entrenado en un archivo para su uso futuro. Este proceso automatiza la clasificación de nuevas imágenes basada en el aprendizaje previo.

In [ ]:

for img_file in os.listdir(positivas_dir):
    img_path = os.path.join(positivas_dir, img_file)
    img = cv.imread(img_path)
    if img is not None:
        features = extract_features(img)
        data.append(features)
        labels.append(1)

for img_file in os.listdir(negativas_dir):
    img_path = os.path.join(negativas_dir, img_file)
    img = cv.imread(img_path)
    if img is not None:
        features = extract_features(img)
        data.append(features)
        labels.append(0) 

data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, random_state=80)

svm_model = svm.SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

with open('wally_svm_model2.pkl', 'wb') as f:
    pickle.dump(svm_model, f)
print("Modelo SVM guardado como 'wally_svm_model2.pkl'")

# Detección de objetos con ventanas deslizantes y HOG 

Desarrollé un script para detectar objetos utilizando el descriptor HOG (Histogram of Oriented Gradients) en OpenCV. Primero, definí una función para extraer características HOG de las imágenes. Luego, apliqué un enfoque de ventana deslizante sobre una imagen específica para identificar regiones potenciales donde el objeto podría estar presente. Cada región se procesó para extraer características y se clasificó usando un modelo SVM previamente entrenado. Utilicé la técnica de supresión de no máximos para reducir múltiples detecciones a la más probable. Finalmente, las detecciones confirmadas se marcaron en la imagen original, indicando la presencia del objeto con rectángulos y etiquetas.

In [ ]:
    import cv2 as cv
    import numpy as np
    import pickle
    import os

def extract_features(img, size=(64, 128)):
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        resized = cv.resize(gray, size)
        hog = cv.HOGDescriptor()
        h = hog.compute(resized)
        return h.flatten()
    
    image_path = r'C:\Users\Janin\Documents\IA\datasetWaldo\img\w.png'
    image = cv.imread(image_path)
    
   
    window_size = (100, 100) 
    step_size = 150
    
    
    found = False

    detections = []
    
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            sub_img = image[y:y + window_size[1], x:x + window_size[0]]
            features = extract_features(sub_img)
            prediction = svm_model.predict([features])
            if prediction == 1:  # Wally encontrado
                detections.append((x, y, window_size[0], window_size[1]))
    
    def non_max_suppression(boxes, overlapThresh):
        if len(boxes) == 0:
            return []
    
        boxes = np.array(boxes)
        pick = []
    
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 0] + boxes[:, 2]
        y2 = boxes[:, 1] + boxes[:, 3]
    
        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)
    
        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)
    
            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])
    
            w = np.maximum(0, xx1 - xx2 + 1)
            h = np.maximum(0, yy1 - yy2 + 1)
    
            overlap = (w * h) / area[idxs[:last]]
    
            idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
    
        return boxes[pick].astype(int)
    
    detections = non_max_suppression(detections, overlapThresh=0.3)
    
    for (x, y, w, h) in detections:
        cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.putText(image, 'Wally', (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        found = True
    
    if not found:
        print("Wally no encontrado en la imagen. :c")
    
    height, width, _ = image.shape
    
    resize_factor = 0.7  
    new_width = int(width * resize_factor)
    new_height = int(height * resize_factor)
    
    cv.namedWindow('Donde esta Wally', cv.WINDOW_NORMAL)
    cv.resizeWindow('Donde esta Wally', new_width, new_height)
    cv.imshow('Donde esta Wally', image)
    cv.waitKey(0)
    cv.destroyAllWindows()